In [1]:
import numpy as np
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

c:\Users\Apm1a\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
def extract_number_simple(image):
    I=np.asarray(image)
    Mask=np.zeros(I.shape)
    Mask[I<254]=255
    Mask=np.uint8(Mask)
    Mask=255-Mask
    im2=Image.fromarray(Mask)
    number=pytesseract.image_to_string(im2,config='--psm 10')
    return number

In [5]:
import requests
import re
def obtener_texto(nombre_carta):
    url = f'https://api.magicthegathering.io/v1/cards?name={nombre_carta}'
    
    try:
        respuesta = requests.get(url)
        respuesta.raise_for_status()  # Verifica si hubo algún error en la solicitud

        datos_carta = respuesta.json()
        if 'cards' in datos_carta and datos_carta['cards']:
            # Supongamos que estamos interesados en el texto de la primera carta encontrada
            texto_carta = datos_carta['cards'][0].get('text', 'No hay texto disponible para esta carta.')
            return texto_carta
        else:
            return 'No se encontraron cartas con ese nombre.'

    except requests.exceptions.RequestException as e:
        return f'Error en la solicitud: {e}'
    
def extraer_letra(expresion):
    # Utilizar expresiones regulares para encontrar la letra entre corchetes
    match = re.search(r'\{([A-Za-z])\}', expresion)
    
    # Verificar si se encontró una coincidencia y devolver la letra
    if match:
        return match.group(1)
    else:
        return None
    
def land_lista():
    d={}
    nombre_archivo = "lands.txt"
    try:
        with open(nombre_archivo, 'r') as archivo:
            lineas = archivo.readlines()
            for linea in lineas[1:]:
                key=' '.join(linea.split(' ')[1:]).replace('\n','')
                d[key]=extraer_letra(re.findall(r'Add \{.*?\}', obtener_texto(key))[0])
        return d

    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no se encontró.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
        
        
def lista():
    l=[]
    nombre_archivo = "list.txt"
    try:
        with open(nombre_archivo, 'r') as archivo:
            lineas = archivo.readlines()
            for linea in lineas:
                key=' '.join(linea.split(' ')[1:]).replace('\n','')
                l.append(key)
  
        return l

    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no se encontró.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
        


        



In [6]:
import cv2
import Levenshtein
import numpy as np
import pandas as pd
def extract_name(image):
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    texto_extraido = pytesseract.image_to_string(gris, config='--psm 10')
    d=lista()
    minimum=999
    for key in d:
        distance=Levenshtein.distance(texto_extraido,key)
        if distance<minimum:
            minimum=distance
            sol=key
    return sol

def extraer_lands(imagen, active=False):

    imagen=imagen[540:700,300:1200]
    #cv2.imshow('Frame',imagen )
    #cv2.waitKey(0)
    # Convierte la imagen a escala de grises
    grises = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    
    # Aplica un umbral para identificar los objetos
    if not active:
        _, umbral = cv2.threshold(grises, 80, 255, cv2.THRESH_BINARY)
    else:
        _, umbral = cv2.threshold(grises, 185, 255, cv2.THRESH_BINARY)
    # Encuentra los contornos en la imagen umbralizada
    #contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    # Crea una máscara negra del mismo tamaño que la imagen original
    mascara = np.zeros_like(imagen)

    # Dibuja los contornos en la máscara
    cv2.drawContours(mascara, contornos, -1, (0,0, 255), thickness=2)

    # Aplica la máscara a la imagen original
    resultado = cv2.bitwise_and(imagen, mascara)
    
    # Extrae cada imagen dentro de los contornos
    imagenes_contornos = []
    lbl=[]
    for contorno in contornos:
        # Encuentra el cuadro delimitador para cada contorno
        (x, y, w, h) = cv2.boundingRect(contorno)
        # Filtra por tamaño mínimo
        if w * h >= 100 and h+6>120 and w+6>85:
            extra_lands=(w-88)//22
            if not active:#No eres el jugador con prioridad
                # Recorta la región de interés (ROI) de la imagen original
                roi = imagen[y-3:y+h+3,x-3:x+w+3]
                imagenes_contornos.append(roi[5:18,6:102])
                #imagen_procesada = Image.fromarray(roi)
                #imagen_procesada = preparar_imagen_para_modelo(imagen_procesada)
                clase_predicha = extract_name(roi[5:18,6:102])             


                for iterations in range(extra_lands+1):
                    lbl.append(clase_predicha)


            else:
            
                roi = imagen[y:y+h,x:x+w]
                imagenes_contornos.append(roi[5:20,6:102])
                #imagen_procesada = Image.fromarray(roi)
                #imagen_procesada = preparar_imagen_para_modelo(imagen_procesada)  
                clase_predicha = extract_name(roi[5:20,6:102])
                #real_class=clases[clase_predicha[0]]               
                for iterations in range(extra_lands+1):
                    lbl.append(clase_predicha)
                

            
            
    return imagenes_contornos,lbl

In [7]:
def avaiable_mana(lbl):#Conversión a forma vectorial del maná disponible
    cards_text=land_lista()
    df = pd.DataFrame([[0, 0, 0, 0, 0, 0]], columns=['R', 'W', 'U', 'B', 'G', 'C'])
    for i in lbl:
        df[cards_text[i]]=df[cards_text[i]]+1
    return df

In [8]:
def extraer_creatures_player(imagen, active=False):

    # Lee la imagen
    #imagen = cv2.imread(imagen_path)
    #[540:700,300:1200]
    imagen=imagen[380:550,300:1200]


    # Convierte la imagen a escala de grises
    grises = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    
    # Aplica un umbral para identificar los objetos
    if not active:
        _, umbral = cv2.threshold(grises, 80, 255, cv2.THRESH_BINARY)
    else:
        _, umbral = cv2.threshold(grises, 185, 255, cv2.THRESH_BINARY)
    # Encuentra los contornos en la imagen umbralizada
    #contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    # Crea una máscara negra del mismo tamaño que la imagen original
    mascara = np.zeros_like(imagen)

    # Dibuja los contornos en la máscara
    cv2.drawContours(mascara, contornos, -1, (0,0, 255), thickness=2)

    # Aplica la máscara a la imagen original
    resultado = cv2.bitwise_and(imagen, mascara)
    
    # Extrae cada imagen dentro de los contornos
    imagenes_contornos = []
    lbl=[]
    for contorno in contornos:
        # Encuentra el cuadro delimitador para cada contorno
        (x, y, w, h) = cv2.boundingRect(contorno)
        # Filtra por tamaño mínimo
        if w * h >= 100 and h+6>120 and w+6>85:
            extra_lands=(w-88)//22
            if not active:#No eres el jugador con prioridad
                # Recorta la región de interés (ROI) de la imagen original
                roi = imagen[y-3:y+h+3,x-3:x+w+3]
                imagenes_contornos.append(roi[5:18,6:102])
                #imagen_procesada = Image.fromarray(roi)
                #imagen_procesada = preparar_imagen_para_modelo(imagen_procesada)
                clase_predicha = extract_name(roi[5:18,6:102])             


                for iterations in range(extra_lands+1):
                    lbl.append(clase_predicha)


            else:
            
                roi = imagen[y:y+h,x:x+w]
                imagenes_contornos.append(roi[5:20,6:102])
                #imagen_procesada = Image.fromarray(roi)
                #imagen_procesada = preparar_imagen_para_modelo(imagen_procesada)  
                clase_predicha = extract_name(roi[5:20,6:102])
                #real_class=clases[clase_predicha[0]]               
                for iterations in range(extra_lands+1):
                    lbl.append(clase_predicha)
                

            
            
    return imagenes_contornos,lbl

In [9]:
def extract_number(image):
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    texto_extraido = pytesseract.image_to_string(gris, config='--psm 10')
    # Expresión regular para encontrar números fraccionarios
    expresion_regular = r'\b\d+/\d+\b'

    # Buscar coincidencias en el texto
    result = re.findall(expresion_regular, texto_extraido)
    if result!=[]:
        return result[0]
    else:
        return ''

In [10]:
def extraer_creatures_player(imagen, active=False):

    # Lee la imagen
    #imagen = cv2.imread(imagen_path)
    imagen2=imagen[380:536,300:1200]
    imagen=imagen[380:529,300:1200]
    #cv2.imshow('Frame',imagen )
    #cv2.waitKey(0)
    # Convierte la imagen a escala de grises
    grises = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    
    # Aplica un umbral para identificar los objetos
    if not active:
        _, umbral = cv2.threshold(grises, 80, 255, cv2.THRESH_BINARY)
    else:
        _, umbral = cv2.threshold(grises, 170, 255, cv2.THRESH_BINARY)
    # Encuentra los contornos en la imagen umbralizada
    #contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    # Crea una máscara negra del mismo tamaño que la imagen original
    mascara = np.zeros_like(imagen)

    # Dibuja los contornos en la máscara
    cv2.drawContours(mascara, contornos, -1, (0,0, 255), thickness=2)

    # Aplica la máscara a la imagen original
    resultado = cv2.bitwise_and(imagen, mascara)
    # Extrae cada imagen dentro de los contornos
    stats = []
    for contorno in contornos:

        # Encuentra el cuadro delimitador para cada contorno
        (x, y, w, h) = cv2.boundingRect(contorno)
        # Filtra por tamaño mínimo
        if w>86 and h>40:
            roi = imagen[y:y+h,x:x+w]
            
            try:                
                height, width = roi.shape[:2]
                corner = roi[height-11:height, width-25:width]
                statistics=extract_number(corner)
                if statistics!='':
                    stats.append(statistics)
            except:pass

    return stats

In [11]:
def extraer_creatures_opponent(imagen, active=False):

    # Lee la imagen
    #imagen = cv2.imread(imagen_path)
    imagen=imagen[215:370,300:1200]
    # Convierte la imagen a escala de grises
    grises = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    
    # Aplica un umbral para identificar los objetos
    if not active:
        _, umbral = cv2.threshold(grises, 80, 255, cv2.THRESH_BINARY)
    else:
        _, umbral = cv2.threshold(grises, 170, 255, cv2.THRESH_BINARY)
    # Encuentra los contornos en la imagen umbralizada
    #contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    # Crea una máscara negra del mismo tamaño que la imagen original
    mascara = np.zeros_like(imagen)

    # Dibuja los contornos en la máscara
    cv2.drawContours(mascara, contornos, -1, (0,0, 255), thickness=2)
    # Aplica la máscara a la imagen original
    resultado = cv2.bitwise_and(imagen, mascara)

    # Extrae cada imagen dentro de los contornos
    stats = []
    for contorno in contornos:
        # Encuentra el cuadro delimitador para cada contorno
        (x, y, w, h) = cv2.boundingRect(contorno)
        # Filtra por tamaño mínimo
        if w>86 and h>40:
            roi = imagen[y:y+h,x:x+w]
            
            try:                
                height, width = roi.shape[:2]
                corner = roi[height-11:height, width-25:width]
                statistics=extract_number(corner)
                if statistics!='':
                    stats.append(statistics)
            except:pass

    return stats

In [12]:
def extract_combat_stats(pstats,ostats):#Recibe como parametros las stats de combate del player y las del oponente
    df = pd.DataFrame([[0, 0, 0, 0]], columns=['PS', 'PT', 'OS', 'OT'])
    for creature in pstats:
        aux=creature.split('/')
        df['PS']=df['PS']+int(aux[0])
        df['PT']=df['PT']+int(aux[1])
    for creature in ostats:
        aux=creature.split('/')
        df['OS']=df['OS']+int(aux[0])
        df['OT']=df['OT']+int(aux[1])

In [13]:
def extract_name_hand(image):
    texto_extraido = pytesseract.image_to_string(image, config='--psm 9')
    d=lista()
    minimum=999
    for key in d:
        distance=Levenshtein.distance(texto_extraido,key)
        if distance<minimum:
            minimum=distance
            sol=key
    return sol

In [14]:
def dict_code_list():
    d={}
    counter=0
    for i in lista():
        d[i]=counter
        counter+=1
        
    return d

In [15]:
def code_hand(imagen,active=False):
    cards=[]
    # Lee la imagen
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    # Convierte la imagen a escala de grises
    imagen=imagen[768:798,168:1725]


    index=0
    for i in range(0,7):    
        card=imagen[:,index:index+185]

        píxeles_en_rango = np.all((card >= 0) & (card <= 27))
        if not píxeles_en_rango:
            cards.append(extract_name_hand(imagen[:,index:index+186]))
            
        index+=186
    
    return cards

In [16]:
def read_log(imagen):
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    imagen=imagen[90:1000,1728:1890]
    text=pytesseract.image_to_string(imagen,lang='eng', config='--psm 3')
    #patron = r"\(.*?\)"
    #text= re.sub(patron, "", text,flags=re.DOTALL)
    

    #text=text.split('.')
    #l=[]
    #for i in text:
        
        #for x in i.split('\n\n'):
            #l.append(x)
    #return l
    return text

In [17]:
def eliminar_no_letras_espacios(cadena):
    # Se ajusta la condición para permitir letras y espacios.
    cadena_filtrada = ''.join(caracter for caracter in cadena if caracter.isalpha() or caracter.isspace())
    return cadena_filtrada

In [18]:
def eliminar_primera_frase(cadena):
    indice_tres_espacios = cadena.find("   ")  # Buscar el índice de los primeros tres espacios seguidos
    if indice_tres_espacios != -1:
        return cadena[indice_tres_espacios + 3:]  # Devolver todo después de los tres espacios
    return cadena  # Si no hay tres espacios seguidos, devolver la cadena original

In [19]:
def extraer_informacion_adicional(cadena1, cadena2, umbral=5):
    #print('Iteración')
    #print('\n')
    
    cadena1 = eliminar_no_letras_espacios(cadena1.replace('\n', ' ').replace('Apmi','Apm'))
    cadena2 = eliminar_no_letras_espacios(cadena2.replace('\n', ' ').replace('Apmi','Apm'))
    cadena1 = eliminar_primera_frase(cadena1)
    cadena2 = eliminar_primera_frase(cadena2)
    #print(cadena2) 
    indice_inicio_subcadena = None
    mejor_distancia = float('inf')
    mejor_i = None  # Para almacenar el índice de la mejor coincidencia
    
    longitud_minima = min(len(cadena1), len(cadena2))
    for i in range(longitud_minima, 0, -1):  # Invertimos el bucle para empezar por la subcadena más larga
        subcadena1 = cadena1[-i:]
        subcadena2 = cadena2[:i]
        distancia = Levenshtein.distance(subcadena1, subcadena2)
        
        # Si la distancia es menor o igual al umbral, actualizamos y paramos, buscando la coincidencia más larga posible
        if distancia <= umbral:
            mejor_distancia = distancia
            mejor_i = i
            break  # Nos detenemos al encontrar la coincidencia más larga dentro del umbral
    
    # Extraemos la información adicional basándonos en la mejor coincidencia encontrada
    if mejor_i is not None and mejor_distancia <= umbral:
        informacion_adicional = cadena2[mejor_i:]
        return informacion_adicional.strip()  # Eliminamos espacios adicionales al inicio y al final
    else:
        return cadena2.strip()



In [20]:
def equal_images(im1,im2):
    imagen1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    imagen2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    imagen1=imagen1[90:1000,1732:1903]
    imagen2=imagen2[90:1000,1732:1903]
    # Verificar si las imágenes tienen el mismo tamaño y canales
    
    # Calcular la diferencia entre las imágenes
    diferencia = cv2.subtract(imagen1, imagen2)        
    # Si no hay diferencia, todas las entradas serán cero    
    resultado = not np.any(diferencia)
    diferencia=np.count_nonzero(diferencia)
    if diferencia>3000:        
        cadena2=pytesseract.image_to_string(imagen2,lang='eng', config='--psm 3')
        cadena2 = eliminar_no_letras_espacios(cadena2.replace('\n', ' ').replace('Apmi','Apm'))
        return cadena2
    else:
        return False

In [21]:
def is_empty(cadena):
    cadena_filtrada = ''.join(caracter for caracter in cadena if caracter.isalpha())
    return cadena_filtrada

In [22]:
def calcular_similitud(cadena1, cadena2, umbral):
    distancia = Levenshtein.distance(cadena1, cadena2)
    return distancia>umbral

In [23]:
def active_player(image):        
    imagen = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    patch1=imagen[390:400,185:220]
    patch2=imagen[300:310,185:220]
    return patch1.mean()>patch2.mean()

In [24]:
def error_recognision(cadena):

    sol=''
    for caracter in cadena:
        if caracter.isnumeric():
            sol+=caracter
            
    if sol!='':
        return int(sol)
    else:
        return 0


In [25]:
columns=list(dict_code_list())
columns.append('log')
for x in ['R', 'W', 'U', 'B', 'G', 'C', 'PS', 'PT', 'OS', 'OT','op_library_cards','op_hand_cards','op_life','op_graveyard_card','op_exiled_card','library_cards','hand_cards','life','graveyard_card','exiled_card']:
    columns.append(x)
df = pd.DataFrame(columns=columns)


# Abre el video
cap = cv2.VideoCapture('G1.mp4')

# Revisa si el video se abrió correctamente
if not cap.isOpened():
    print("Error: No se pudo abrir el video.")
    exit()
    

ret, ant = cap.read()
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
initial=True
iteracion=0
# Recorre el video
while True:
    
    # Obtiene el número del frame actual
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    
    # Calcula el porcentaje del procesado
    progress_percentage = (current_frame / total_frames) * 100    
    
    
    ret, frame = cap.read()

    if not ret:
        print("No se pudo recibir el frame. Salir.")
        break
    
    log_info=equal_images(ant,frame)
    ant=frame 
    #Capturamos el log
    if log_info!=False and is_empty(log_info)!='':
        if initial:
            #dejamos algo de margen
            for margin in range(0,5):
                ret, frame = cap.read()
                ant=frame
                

            #inicializamos una nueva fila de 0s
            df.loc[iteracion] = {col: 0 for col in df.columns}
            
            #cargamos los segmentos de la imagen donde tenemos cada información relevante
            op_library_cards=frame[28:50,83:112]
            op_hand_cards=frame[28:50,114:131]
            op_life=frame[95:155,70:138]
            op_graveyard_card=frame[189:216,20:50]
            op_exiled_card=frame[191:215,51:78]

            library_cards=frame[577:598,83:111]
            hand_cards=frame[578:600,114:131]
            life=frame[655:705,70:138]
            graveyard_card=frame[746:762,27:42]
            exiled_card=frame[745:763,58:71]
        

            #alamacenamos y leemos la información
            df.at[iteracion, 'op_library_cards'] = error_recognision(extract_number_simple(op_library_cards))
            df.at[iteracion, 'op_hand_cards'] = error_recognision(extract_number_simple(op_hand_cards))
            df.at[iteracion, 'op_life'] = error_recognision(extract_number_simple(op_life))
            df.at[iteracion, 'op_graveyard_card'] = error_recognision(extract_number_simple(op_graveyard_card))
            df.at[iteracion, 'op_exiled_card'] = error_recognision(extract_number_simple(op_exiled_card))


            df.at[iteracion, 'library_cards'] = error_recognision(extract_number_simple(library_cards))
            df.at[iteracion, 'hand_cards'] = error_recognision(extract_number_simple(hand_cards))
            df.at[iteracion, 'life'] = error_recognision(extract_number_simple(life))
            df.at[iteracion, 'graveyard_card'] = error_recognision(extract_number_simple(graveyard_card))
            df.at[iteracion, 'exiled_card'] = error_recognision(extract_number_simple(exiled_card))
            
            #obtenemos el jugado activo
            active=active_player(frame)
            #guardamos el log y lo actualizamos
            df.at[iteracion,'log']= log_info
            last_log=log_info
            initial=False
            
            #extraemos los manás que tenemos
            im,lbl=extraer_lands(frame,active)
            cards_text=land_lista()
            for i in lbl:
                df.at[iteracion,cards_text[i]]=df.at[iteracion,cards_text[i]]+1
                
            
            #ahora vamos a extraer un resumen de la fase de combate
            pstats=extraer_creatures_player(frame,active)
            ostats=extraer_creatures_opponent(frame,active)
            for creature in pstats:
                aux=creature.split('/')
                df['PS']=df['PS']+int(aux[0])
                df['PT']=df['PT']+int(aux[1])
            for creature in ostats:
                aux=creature.split('/')
                df['OS']=df['OS']+int(aux[0])
                df['OT']=df['OT']+int(aux[1])
            
            #codigicamos y almacenamos la mano
            hand=code_hand(frame,active)

            for card in hand:
                if card in df.columns:
                    df.at[iteracion,card]+=1
            iteracion+=1
            
        #haremos lo mismo pero sin tener en cuenta que es la primera iteración, por lo tanto buscaremos aquellos logs que tengan una diferencia significativa con el anterior
        elif calcular_similitud(log_info,last_log,8):
            
            
            
            for margin in range(0,5):
                ret, frame = cap.read()
                ant=frame
                

        
            df.loc[iteracion] = {col: 0 for col in df.columns}
            active=active_player(frame)
            #cargamos los segmentos de la imagen donde tenemos cada información relevante
            op_library_cards=frame[28:50,83:112]
            op_hand_cards=frame[28:50,114:131]
            op_life=frame[95:155,70:138]
            op_graveyard_card=frame[189:216,20:50]
            op_exiled_card=frame[191:215,51:78]

            library_cards=frame[577:598,83:111]
            hand_cards=frame[578:600,114:131]
            life=frame[655:705,70:138]
            graveyard_card=frame[746:762,27:42]
            exiled_card=frame[745:763,58:71]
        

            #alamacenamos y leemos la información
            df.at[iteracion, 'op_library_cards'] = error_recognision(extract_number_simple(op_library_cards))
            df.at[iteracion, 'op_hand_cards'] = error_recognision(extract_number_simple(op_hand_cards))
            df.at[iteracion, 'op_life'] = error_recognision(extract_number_simple(op_life))
            df.at[iteracion, 'op_graveyard_card'] = error_recognision(extract_number_simple(op_graveyard_card))
            df.at[iteracion, 'op_exiled_card'] = error_recognision(extract_number_simple(op_exiled_card))


            df.at[iteracion, 'library_cards'] = error_recognision(extract_number_simple(library_cards))
            df.at[iteracion, 'hand_cards'] = error_recognision(extract_number_simple(hand_cards))
            df.at[iteracion, 'life'] = error_recognision(extract_number_simple(life))
            df.at[iteracion, 'graveyard_card'] = error_recognision(extract_number_simple(graveyard_card))
            df.at[iteracion, 'exiled_card'] = error_recognision(extract_number_simple(exiled_card))
            
            
            
            
            
            df.at[iteracion,'log']=extraer_informacion_adicional(last_log,log_info,5)
            last_log=log_info
            im,lbl=extraer_lands(frame,True)
            
            cards_text=land_lista()
            for i in lbl:
                df.at[iteracion,cards_text[i]]=df.at[iteracion,cards_text[i]]+1  


            pstats=extraer_creatures_player(frame,active)
            ostats=extraer_creatures_opponent(frame,active)
            for creature in pstats:
                aux=creature.split('/')
                df['PS']=df['PS']+int(aux[0])
                df['PT']=df['PT']+int(aux[1])
            for creature in ostats:
                aux=creature.split('/')
                df['OS']=df['OS']+int(aux[0])
                df['OT']=df['OT']+int(aux[1])



            hand=code_hand(frame,active)

            for card in hand:
                if card in df.columns:
                    df.at[iteracion,card]+=1
            iteracion+=1
    
    #print(f"Procesado: {progress_percentage:.2f}%")
        
    


        # Muestra el frame actual

    
    # Espera por una tecla para mostrar el siguiente frame
    #if cv2.waitKey(0) & 0xFF == ord('q'):
    #    break
    
    #cv2.imshow('Frame', ant[90:1000,1732:1903])
    #cv2.imshow('Frameact',frame[90:1000,1732:1903])


    
    
    # Si ret es False, el video ha terminado o hay un error
    

# Cuando todo está hecho, libera el capturador
cap.release()
cv2.destroyAllWindows()

No se pudo recibir el frame. Salir.


In [26]:
df

,Chain Lightning,End the Festivities,Experimental Synthesizer,Fireblast,Galvanic Blast,Goblin Blast-Runner,Goblin Bushwhacker,Goblin Tomb Raider,Great Furnace,Implement of Combustion,...,op_library_cards,op_hand_cards,op_life,op_graveyard_card,op_exiled_card,library_cards,hand_cards,life,graveyard_card,exiled_card
0,0,0,2,0,1,0,0,0,0,0,...,54,7,20,0,0,54,6,20,0,0
1,0,0,0,2,2,0,0,0,0,0,...,54,0,2,0,0,54,6,2,0,0
2,0,0,1,1,2,0,0,1,0,0,...,54,5,0,0,0,53,7,20,0,0
3,0,0,2,1,2,0,0,1,0,0,...,54,5,2,0,0,53,0,0,0,0
4,0,0,2,0,2,0,0,1,0,0,...,54,5,2,0,0,53,5,0,0,0
5,0,0,0,2,2,0,0,1,0,0,...,52,5,26,0,0,53,5,2,0,0
6,0,0,0,1,2,0,0,1,0,0,...,5,5,0,0,0,53,5,27,0,0
7,0,0,1,0,2,0,0,1,0,0,...,53,5,0,0,0,53,5,0,0,0
8,0,0,1,0,2,0,0,1,0,0,...,55,5,0,0,0,5,5,20,0,0
9,0,0,0,0,1,0,0,1,0,0,...,52,6,0,0,0,53,5,0,0,0


In [27]:
df2=df

In [28]:
def procesar_logs(logs):
    first=True
    log=[]
    for log_info in logs:
        if first:
            log.append(extraer_informacion_adicional(logs[-1],log_info,5))
    return log

In [29]:
import random
def son_numeros_cercanos(valor1, valor2, tolerancia):
    return abs(valor1 - valor2) <= tolerancia

def revisar_numeros_cercanos(df, tolerancia,columna):
    for i in range(1, len(df) - 1):
        anterior = df.iloc[i - 1][columna] # Cambia 'columna' por el nombre de la columna que quieres revisar
        actual = df.iloc[i][columna]
        if not son_numeros_cercanos(actual, anterior, tolerancia):
            #valor_menor, valor_mayor = ordenar_valores(anterior,siguiente)
            #valor=random.randint( valor_menor,valor_mayor)

            df.at[i, columna] = anterior
    return df

In [30]:
for columna,tolerancia in zip(['op_library_cards','op_hand_cards','op_life','op_graveyard_card','op_exiled_card','library_cards','hand_cards','life','graveyard_card','exiled_card'],[1,2,5,1,1,1,2,5,1,1]):

    df2=revisar_numeros_cercanos(df2,tolerancia,columna)

In [31]:
df

,Chain Lightning,End the Festivities,Experimental Synthesizer,Fireblast,Galvanic Blast,Goblin Blast-Runner,Goblin Bushwhacker,Goblin Tomb Raider,Great Furnace,Implement of Combustion,...,op_library_cards,op_hand_cards,op_life,op_graveyard_card,op_exiled_card,library_cards,hand_cards,life,graveyard_card,exiled_card
0,0,0,2,0,1,0,0,0,0,0,...,54,7,20,0,0,54,6,20,0,0
1,0,0,0,2,2,0,0,0,0,0,...,54,7,20,0,0,54,6,20,0,0
2,0,0,1,1,2,0,0,1,0,0,...,54,5,20,0,0,53,7,20,0,0
3,0,0,2,1,2,0,0,1,0,0,...,54,5,20,0,0,53,7,20,0,0
4,0,0,2,0,2,0,0,1,0,0,...,54,5,20,0,0,53,5,20,0,0
5,0,0,0,2,2,0,0,1,0,0,...,54,5,20,0,0,53,5,20,0,0
6,0,0,0,1,2,0,0,1,0,0,...,54,5,20,0,0,53,5,20,0,0
7,0,0,1,0,2,0,0,1,0,0,...,53,5,20,0,0,53,5,20,0,0
8,0,0,1,0,2,0,0,1,0,0,...,53,5,20,0,0,53,5,20,0,0
9,0,0,0,0,1,0,0,1,0,0,...,52,6,20,0,0,53,5,20,0,0


In [32]:
df.to_csv('g1.csv', index=False)